In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import sys

sys.path.append('../src')

In [3]:
import torch
import torchvision
import trainer
import dataset_masker as dataset
import skimage.transform
import model_masker as model

import utils

anchors = [[1.0, 1.0]]
mask_size = [15, 15]

train_dataset = dataset.DataSet('../data/', mode='train', anchors=anchors, mask_size=mask_size)
valid_dataset = dataset.DataSet('../data/', mode='valid', anchors=anchors, mask_size=mask_size)

In [2]:
# Basic collator function
def my_collator(args):
    res = None
    for arg in args:
        if res is None:
            res = []
            for item in arg:
                res.append([])
        for index in range(len(arg)):
            res[index].append(arg[index])
    
    return res

train_loader = torch.utils.data.DataLoader(train_dataset, 
                                           batch_size=4, 
                                           shuffle=True,
                                           num_workers=2,
                                           drop_last=True,)
#                                            collate_fn=my_collator)

valid_loader = torch.utils.data.DataLoader(valid_dataset,
                                           batch_size=4,
                                           shuffle=True,
                                           num_workers=2,
                                           drop_last=False,)
#                                            collate_fn=my_collator)

net = model.Network(len(anchors), mask_size).float().cuda()
input = torch.autograd.Variable(torch.zeros(10, 3, 128, 128).float().cuda())
result = net.forward([input])


socket = model.Socket(net, anchors)

pretrain_modules = torch.nn.ModuleList([socket.model.up_bn1, 
                                        socket.model.up_bn2, 
                                        socket.model.up_bn3,
                                        socket.model.up1, 
                                        socket.model.up2, 
                                        socket.model.up3,
                                        socket.model.detector]).parameters()
# train_moduels = net.parameters()

pretrain_optimizer = torch.optim.Adam(pretrain_modules, lr=3.0e-4)
my_pretrainer = trainer.Trainer(socket, pretrain_optimizer)

for index in range(1000):
#     print("Epoch", index)
    print("Epoch", index)
    print("Training ... ", end='')
    loss = my_pretrainer.train(train_loader)
    print("Done")
    print("Validating ... ", end='')
    train_metrics = my_pretrainer.validate(train_loader)
    valid_metrics = my_pretrainer.validate(valid_loader)
    print("Done")
    print('; '.join([str(x) for x in train_metrics + valid_metrics]))
    
#     my_pretrainer.validate()

my_pretrainer.make_checkpoint('checkpoint_pretrained.t7')

train_modules = socket.model.parameters()

train_optimizer = torch.optim.Adam(train_modules, lr=3.0e-4)
my_trainer = trainer.Trainer(socket, train_optimizer)

for index in range(1000):
#     print("Epoch", index)
    print("Epoch", index)
    print("Training ... ", end='')
    loss = my_pretrainer.train(train_loader)
    print("Done")
    print("Validating ... ", end='')
    train_metrics = my_pretrainer.validate(train_loader)
    valid_metrics = my_pretrainer.validate(valid_loader)
    print("Done")
    print('; '.join([str(x) for x in train_metrics + valid_metrics]))

my_trainer.make_checkpoint('checkpoint.t7')

In [ ]:
def bbox_iou(bbox1, bbox2):
    left = max(bbox1[0], bbox2[0])
    top = max(bbox1[1], bbox2[1])
    right = min(bbox1[2], bbox2[2])
    bottom = min(bbox1[3], bbox2[3])
    
    intersection = max((right - left) * (top - bottom), 0)
    union = (bbox1[2] - bbox1[0]) * (bbox1[3] - bbox1[1])
    union += (bbox2[2] - bbox2[0]) * (bbox2[3] - bbox2[1])
    
    return intersection / union



# Дальше надо сделать формирование таргета и начинать обучение при помощи тренера

In [ ]:
for epoch in range(100):
    for res in train_loader:
        print(res)
        break
#         print(batch)
        #print(images)
        print(targets[0], '<- bboxes')
        print(targets[1], '<- masks')
#         print(targets[2], '<- wtf')
#         print(targets, '<- targets')
        break
    break
#         print(len(images), len(targets), end=' ')

In [ ]:
%pylab inline
def plotter(img, masks, alpha=0.5):
#     figure(figsize=(20, 20))
    imshow(img)
    print(img.shape)
    overlay = numpy.zeros(img.shape[:2] + (4,))
    overlay[:, :, 0] = 1.0
    
    for mask in masks:
        bbox = mask[0]
        
        left = round(bbox[0] * (img.shape[0] - 1))
        top = round(bbox[1] * (img.shape[1] - 1))
        right = round(bbox[2] * (img.shape[0] - 1))
        bottom = round(bbox[3] * (img.shape[1] - 1))
        
#         print(mask[1])
        if (right - left) < 0.5 or (bottom - top) < 0.5:
            continue
        mask_tensor = skimage.transform.resize(
            mask[1], 
            [round(right) - round(left) + 1, round(bottom) - round(top) + 1], 
            mode='reflect')
        
        plot([top, top], [left, right + 1], 'r')
        plot([bottom + 1, bottom + 1], [left, right + 1], 'r')
        plot([top, bottom + 1], [left, left], 'r')
        plot([top, bottom + 1], [right + 1, right + 1], 'r')
        
        overlay[round(left):round(right) + 1, 
                round(top):round(bottom) + 1, 3] = mask_tensor * alpha
        
    imshow(overlay)
        
        

In [ ]:
import dataset

my_dataset = dataset.DataSet('../data/')
print(len(my_dataset))

index = 0
while True:
    index += 1
    print(index, len(my_dataset), end=' ')
    if index >= len(my_dataset):
        index = 0
    
    img, masks = my_dataset[index]
    
    if img.size(1) != 112:
        break

In [ ]:
import mnist

train_images = mnist.train_images()
train_labels = mnist.train_labels()

print(train_images.shape)
print(train_labels.shape)

In [ ]:
import mnist

In [ ]:
mnist.